In [8]:
!pip install opencv-python numpy Pillow

In [9]:
!pip install diffusers transformers torch

In [10]:
import cv2
import numpy as np
import os
from glob import glob
from PIL import Image, ImageDraw, ImageFont
import uuid

# Placeholder function to simulate image generation (replace with real model)
def generate_images_from_prompt(prompt, num_images=2, width=1280, height=720):
    """
    Simulate generating images from a text prompt.
    Replace this with a real image generation model (e.g., Stable Diffusion).
    Returns a list of file paths to generated images.
    """
    output_paths = []
    try:
        # Try to use a default font; fall back to Pillow's default if unavailable
        try:
            font = ImageFont.truetype("arial.ttf", 50)
        except:
            font = ImageFont.load_default()

        for i in range(num_images):
            # Create a blank image with a colored background
            colors = [(255, 100, 100), (100, 255, 100)]  # Reddish, Greenish
            img = Image.new("RGB", (width, height), color=colors[i % len(colors)])
            draw = ImageDraw.Draw(img)
            # Add text to simulate the prompt
            text = f"{prompt} (Image {i+1})"
            text_bbox = draw.textbbox((0, 0), text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            text_x = (width - text_width) // 2
            text_y = (height - text_height) // 2
            draw.text((text_x, text_y), text, fill=(255, 255, 255), font=font)
            # Save image
            output_path = f"generated_image_{i+1}.png"
            img.save(output_path)
            output_paths.append(output_path)
    except Exception as e:
        print(f"Error generating images: {e}")
        return []
    return output_paths

# Parameters
prompt = "A majestic tiger in a jungle"  # Text prompt
output_file = "tiger_video.mp4"  # Output video file name
duration = 3  # Total video duration in seconds (2-5 seconds)
fps = 24  # Frames per second
width, height = 1280, 720  # Video resolution
transition_duration = 0.5  # Duration of crossfade in seconds
image_folder = "."  # Directory to save generated images

# Debugging: Print current directory
print("Current working directory:", os.getcwd())
print("Saving generated images to:", os.path.abspath(image_folder))

# Generate images from prompt
image_files = generate_images_from_prompt(prompt, num_images=2, width=width, height=height)
print("Generated image files:", image_files)

# Check if enough images were generated
if len(image_files) < 2:
    print("Error: Could not generate at least 2 images. Check prompt or generation function.")
    exit()

# Load and resize images
images = []
for img_path in image_files:
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Error: Failed to load image {img_path}. It may be corrupted or inaccessible.")
        continue
    images.append(cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA))
print(f"Successfully loaded {len(images)} images")

# Check if enough valid images were loaded
if len(images) < 2:
    print("Error: Could not load at least 2 valid images.")
    exit()

# Calculate frames per image
total_frames = int(duration * fps)
frames_per_image = total_frames // len(images)
transition_frames = int(transition_duration * fps)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
if not video.isOpened():
    print("Error: Could not initialize video writer. Try installing FFmpeg and using 'X264' codec.")
    exit()

# Generate video frames
for i in range(total_frames):
    current_image_idx = min(i // frames_per_image, len(images) - 1)
    next_image_idx = min(current_image_idx + 1, len(images) - 1)

    # Calculate transition progress
    frame_in_image = i % frames_per_image
    if frame_in_image < transition_frames and current_image_idx < len(images) - 1:
        # Crossfade transition
        alpha = frame_in_image / transition_frames
        beta = 1 - alpha
        frame = cv2.addWeighted(images[current_image_idx], beta, images[next_image_idx], alpha, 0.0)
    else:
        # Display current image
        frame = images[current_image_idx]

    video.write(frame)

# Release video writer
video.release()

# Clean up generated images (optional)
for img_path in image_files:
    try:
        os.remove(img_path)
    except:
        pass

print(f"Video saved as {output_file}")


Current working directory: c:\Users\SreeKeerthiReddyThat\Desktop\Image-generation\lvm
Saving generated images to: c:\Users\SreeKeerthiReddyThat\Desktop\Image-generation\lvm
Generated image files: ['generated_image_1.png', 'generated_image_2.png']
Successfully loaded 2 images
Video saved as tiger_video.mp4
